In [ ]:
tech_colors= {
    'Transmission lines': "grey",
    'Battery storage' : '#80c944' ,
        'battery charger' : 'darkgreen' ,
    'battery discharger' : 'darkgreen' , 
    'battery' : 'darkgreen' ,        
    'Utility battery' : '#80c944' ,        
    'Home battery storage' : '#ace37f'  ,
    'Distributed battery' : '#ace37f'  ,
    'home battery charger' : '#ace37f'  ,
    'home battery discharger' : '#ace37f' ,    
    'home battery' : '#ace37f' ,    
    'H2 Fuel Cell': '#c251ae',
    'H2 Electrolysis': '#ff29d9',
    'H2 pipeline' : 'plum',
    'SMR': '#870c71',
    'SMR CC': '#4f1745',
    'Sabatier' : '#4f1745',
    'helmeth' : '#4f1745',
    'co2':'linen',
    'co2 vent':'linen',
    'co2 stored':'linen',
    'Electricity distribution grid': 'darkgrey',
    'electricity distribution grid': 'darkgrey',
    'distribution grid': 'darkgrey',
    'uranium': '#ff8c00',
    "onwind": "#235ebc",
    "Onshore wind": "#235ebc",
    "onshore wind": "#235ebc",
    'offwind': "#6895dd",
    'offwind-ac': "#6895dd",
    'Offshore wind': "#6895dd",
    'offshore wind': "#6895dd",
    'offshore wind ac': "#6895dd",
    'offwind-dc': "#74c6f2",
    'offshore wind dc': "#74c6f2",
    "Hydroelectricity": "#08ad97",
    "hydro": "#08ad97",
    "hydro+PHS": "#08ad97",
    "PHS": "#08ad97",
    "Pumped hydro storage": "teal",
    "hydro reservoir": "#08ad97",
    'hydroelectricity': '#08ad97',
    "ror": "#4adbc8",
    "Run of river": "#4adbc8",
    'Solar': "#f9d002",
    'Solar utility': "#f9d002",
    'solar utility': "#f9d002",
    'solar': "#f9d002",
    'solar PV': "#f9d002",
    'Solar rooftop' : 'orange',
    'solar rooftop' : 'orange',
    'Solar distributed' : 'orange',
    'solar distributed' : 'orange',
    'solar thermal': '#ffef60',
    'biomass': 'lime', #'#0c6013',
    'solid biomass': '#06540d',
    'biogas': '#23932d',
    'waste': '#68896b',
    'geothermal': '#ba91b1',
    "OCGT": "#d35050",
    "gas": "#d35050",
        "CCGT": "#d35050",
    "natural gas": "#d35050",
    "CCGT": "#b20101",
    "Nuclear": "rosybrown",
        "nuclear": "rosybrown",
    "coal": "#707070",
    "lignite": "#9e5a01",
    "oil": "#262626",
    "H2 storage": "#ea048a",
    "hydrogen storage": "#ea048a",
    "battery": "#b8ea04",
    "Electric load": "#f9d002",
    "electricity": "#f9d002",
    "lines": "#70af1d",
    "transmission lines": "#70af1d",
    "AC-AC": "#70af1d",
    "AC line": "#70af1d",
    "links": "#8a1caf",
    "HVDC links": "#8a1caf",
    "DC-DC": "#8a1caf",
    "DC link": "#8a1caf",
    'Solar thermal': '#ffef60',
    "Oil and gas to power/heat": "#d35050",
    "Coal": "#707070",
    "Power to liquid": '#4f1745',
    "Power to gas":'#870c71',
    "Power to heat":'maroon',
    "Biomass":'#06540d',
    "Biogas":'#23932d',
    "BEV charger":'rebeccapurple',
    "V2G":'mediumpurple',
    "DAC":'lightcoral',
    "Heat storage":"brown",
     "gas CHP": "orangered",
     "biomass CHP": "darkolivegreen",
     "resistive heater": "indianred",
     "air heat pump": "chocolate",
     "ground heat pump": "sandybrown",
    "Heat storage":"brown",
    "heat pump":'darkslategrey',
    "resistive heater":'maroon',
    "gas boiler": "crimson", 
    "industry": 'black',
    "Fischer-Tropsch":"darkgrey",

}

In [ ]:
opt_name = {
    "Store": "e",
    "Line": "s",
    "Transformer": "s"
}
def calculate_costs(n, label, costs):

    for c in n.iterate_components(n.branch_components|n.controllable_one_port_components^{"Load"}):
        capital_costs = c.df.capital_cost*c.df[opt_name.get(c.name,"p") + "_nom_opt"]
        capital_costs_grouped = capital_costs.groupby(c.df.carrier).sum()

        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=["capital"])
        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(capital_costs_grouped.index.union(costs.index))

        costs.loc[capital_costs_grouped.index, label] = capital_costs_grouped

        if c.name == "Link":
            p = c.pnl.p0.multiply(n.snapshot_weightings.generators, axis=0).sum()
        elif c.name == "Line":
            continue
        elif c.name == "StorageUnit":
            p_all = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0)
            p_all[p_all < 0.] = 0.
            p = p_all.sum()
        else:
            p = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0).sum()

        #correct sequestration cost
        if c.name == "Store":
            items = c.df.index[(c.df.carrier == "co2 stored") & (c.df.marginal_cost <= -100.)]
            c.df.loc[items, "marginal_cost"] = -20.

        marginal_costs = p*c.df.marginal_cost

        marginal_costs_grouped = marginal_costs.groupby(c.df.carrier).sum()

        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=["marginal"])
        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(marginal_costs_grouped.index.union(costs.index))

        costs.loc[marginal_costs_grouped.index,label] = marginal_costs_grouped

    # add back in all hydro
    #costs.loc[("storage_units", "capital", "hydro"),label] = (0.01)*2e6*n.storage_units.loc[n.storage_units.group=="hydro", "p_nom"].sum()
    #costs.loc[("storage_units", "capital", "PHS"),label] = (0.01)*2e6*n.storage_units.loc[n.storage_units.group=="PHS", "p_nom"].sum()
    #costs.loc[("generators", "capital", "ror"),label] = (0.02)*3e6*n.generators.loc[n.generators.group=="ror", "p_nom"].sum()

    return costs

In [ ]:
import pandas as pd
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches


"""
scenarios= {'electricity_sector_only_181_newbase': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-solar+p3-dist1_{}.nc' #Base : 500 €/kW distribution link cost, 0% loss
            ,'electricity_sector_only_181_newbase_norooftop': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-solar+p3-dist1_{}.nc' #no rooftop and no home batteries
            ,'electricity_sector_only_181_newbase_highloss': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-solar+p3-dist2_{}.nc'  #Base with added costs/losses : 1000 €/kW distribution link cost, 10% loss
           , 'electricity_sector_only_181_newbase_highloss_norooftop': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-solar+p3-dist2_{}.nc' #no rooftop and no home batteries 
           , 'sector_coupling_181_newbase_highloss_noroofotp': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-T-H-B-I-solar+p3-dist2_{}.nc' #no rooftop and no home batteries  
           ,'sector_coupling_181_newbase_highloss': 'postnetworks/elec_s_181_lv{}__Co2L0-2H-T-H-B-I-solar+p3-dist2_{}.nc'  #sector-coupled with added costs/losses : 1000 €/kW distribution link cost, 10% loss
           
           } 
"""

main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['','-nohomegen'],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],
          }
scenarios_description= {'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':
                        ['Electricity sector', 'without distributed generation', 'Base distribution grid','A']  ,
            'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':
                        ['Electricity sector', 'with distributed generation', 'Base distribution grid','A']  ,
                        'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2-nohomegen_2030.nc':
                        ['Electricity sector', 'without distributed generation', 'With higher distribution cost/loss','B']  ,
                        'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc':
                        ['Electricity sector', 'with distributed generation', 'With higher distribution cost/loss','B'] ,
                'sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':
                        ['Sector coupled', 'with distributed generation', 'With higher distribution cost/loss','C'] ,
        'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen_2030.nc':
                       ['Sector coupled', 'without distributed generation', 'With higher distribution cost/loss','C'] ,
             'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':
                             ['Sector coupled', 'with distributed generation', 'With higher distribution cost/loss, ','C'] ,
                       } 


transmission_expansion=[1.0] #,'opt']
years = [2030] 





In [ ]:
results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name

for scen in main_scen.keys():
    for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen.format(main_scen[scen][i])
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.storage_units.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                ,pd.Series({'Max rooftop' : n.generators.p_nom_max.filter(like='solar rooftop').sum()})
                 ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
             scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
             scen_costs[scenario] = scen_costs[scenario].rename(index={
                          'battery':'Battery storage','battery charger':'Battery storage','battery discharger':'Battery storage',
                          'home battery':'Home battery storage','home battery charger':'Home battery storage','home battery discharger':'Home battery storage',                                                       
                           'offwind-ac':'Offshore wind','offwind-dc':'Offshore wind', 'onwind':'Onshore wind',
                          'AC':'Transmission lines'  , 'DC':'Transmission lines','ror': 'Run of river',
                          'hydro':'Hydroelectricity' , 'nuclear':'Nuclear','uranium':'Nuclear',
                          'solar':'Solar utility', 'solar rooftop':'Solar rooftop',
                          'electricity distribution grid' : 'Electricity distribution grid',
                           'PHS' : 'Pumped hydro storage', 'H2':'H2 storage',
                           'SMR':'Power to gas'  , 'SMR CC':'Power to gas','oil': 'Oil and gas to power/heat',
                           'Fischer-Tropsch':'Power to liquid', 
                           'Sabatier':'Power to gas', 'helmeth':'Power to gas', 'coal' : 'Coal', 'lignite' :'Coal',
                          })  # co2 vent','co2','co2 stored',
             for ix in scen_costs[scenario].index :   
                if 'resistive heater' in ix or 'heat pump' in ix :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix : 'Power to heat',})
                if  any(w in ix for w in ['gas', 'OCGT', 'CCGT'])  :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Oil and gas to power/heat',})
                if 'biomass' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biomass',})
                if 'solar thermal' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Solar thermal',})
                if 'water tank' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Heat storage',})
                if 'biogas' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biogas',})
                #if 'biogas' in ix :scen_costs[scenario].rename(index={ix:'Biomass',})
                    
             scen_costs[scenario]=scen_costs[scenario].groupby(scen_costs[scenario].index).sum() 

In [ ]:
costs=pd.DataFrame({})

preferred_order = pd.Index(
    [
        "Transmission lines",
        "Electricity distribution grid",
        "Oil and gas to power/heat",
        "Coal",
        "Power to liquid",
        "Power to gas",
        "Biomass",
        "Biogas",
        "BEV charger",
        "V2G",
        "DAC",
        "Power to heat",
        "Heat storage",
        "H2 storage",
        "H2 pipeline",
        "H2 Electrolysis",
        "H2 Fuel Cell",
        "Pumped hydro storage",
        "Hydroelectricity",
        "Run of river",
        "Nuclear",
        "Onshore wind",
        "Offshore wind",
        "Battery storage",
        "Home battery storage",
        "Solar thermal",
        "Solar utility",
        "Solar rooftop",
    ]
)


for  scenario in reversed(scen_costs.keys()) :
    costs[scenario]=pd.DataFrame(scen_costs[scenario].drop(labels =['co2 vent','co2','co2 stored'
                                                                   ]))
                                                                  #  ,'oil','uranium','lignite','coal','gas' ]) ) 
                                                        #   ['SMR', 'SMR CC','Sabatier','co2 vent','co2','co2 stored',
                                                   #    'helmeth', 'oil','uranium','lignite','coal','gas', 'CCGT','OCGT'
                                                  #  ]) ) #dropping all the variables not visible in figure
    

order = preferred_order.intersection(costs[scenario].index).append(
        costs[scenario].index.difference(preferred_order))    
costs=costs.reindex(order)   
costs=costs.drop(labels =['process emissions','process emissions CC','Li ion'])
costs=costs.fillna(0)
results=results.fillna(0)

costs.to_csv('tables/costs.csv')
results.to_csv('tables/results.csv')

In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(3, 1, figsize=(6,20)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

bar_width=0.4
index=np.arange(2) 



main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['-nohomegen',''],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],}  #

costs =costs.rename(index={'Solar rooftop':'Solar distributed',})
costs =costs.rename(index={'Home battery storage':'Distributed battery',})
costs =costs.rename(index={'Battery storage':'Utility battery',})

for i,base in enumerate(main_scen):
    for j, category in enumerate(main_scen[base]):
            scenario=base.format(category) 
            bottom=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
                axs[i].bar(index[j], costs[scenario][component] , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=costs[scenario][component]
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25) 
            if i==0 : axs[i].set_title('without distributed          with distributed\n    generation                           generation\n',fontsize=20, fontname = 'Ubuntu')
            if j==0 : axs[i].set_ylabel(scenarios_description[scenario][0]+'\n'+
                scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
                
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 2.5),ncol=1,
           fontsize=20,frameon=False)

for i ,scen in enumerate (main_scen.keys()):
    fig.text(0.4, 0.25*(3-i), "- "+str(round(scen_costs[scen.format(main_scen[scen][0])].sum()-
                                              scen_costs[scen.format(main_scen[scen][1])].sum(),2))
         +" bn€\n "+str(round(100*((scen_costs[scen.format(main_scen[scen][1])].sum()/
                                     scen_costs[scen.format(main_scen[scen][0])].sum())-1),2))+"%", 
         fontsize=18)
"""
fig.text(0.50, 0.68, "-"+str(round(scen_costs[main_scen[1]+'_norooftop'].sum()-scen_costs[main_scen[1]].sum(),2))
         +"\n-"+str(round(100*((scen_costs[main_scen[1]+'_norooftop'].sum()/scen_costs[main_scen[1]+''].sum())-1),2))+"%", 
         fontsize=18)

fig.text(0.50, 0.32, "-"+str(round(scen_costs[main_scen[2]+'_noroofotp'].sum()-scen_costs[main_scen[2]].sum(),2))
         +"\n-"+str(round(100*((scen_costs[main_scen[2]+'_noroofotp'].sum()/scen_costs[main_scen[2]+''].sum())-1),2))+"%", 
         fontsize=18)
"""
def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


arrow_style = dict(arrowstyle="simple", mutation_scale=15, ec=None)
add_arrow(axs[0], axs[0], (0.35, 0.4), (0.65, 0.4), fc="black", **arrow_style)
add_arrow(axs[1], axs[1], (0.35, 0.45), (0.65, 0.45), fc="black", **arrow_style)
add_arrow(axs[2], axs[2], (0.35, 0.5), (0.65, 0.5), fc="black", **arrow_style)

axs[0].set_yticks(np.arange(0, 301, 50),fontsize=25)
axs[1].set_yticks(np.arange(0, 301, 50),fontsize=25)
axs[2].set_yticks(np.arange(0, 851, 170),fontsize=25)


In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager
import matplotlib.lines as lines

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(figsize=(10,13.33)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

bar_width=0.42
index=np.arange(4) 



main_scen={'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],  #
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],}

scenario_C= 'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc'  
scenario_C_highrooftop= 'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc'
scen_base= scenario_C.format(main_scen[scenario_C][0])
scen_base_highrooftop=scenario_C_highrooftop.format(main_scen[scenario_C_highrooftop][0])
scen_sector_c = scenario_C.format(main_scen[scenario_C][1])               

hatches=['','','/']

scenario_format= {'1':[scen_base, scen_sector_c, scen_sector_c_highrooftop],}

for i,base in enumerate(scenario_format):
    for j, category in enumerate(scenario_format[base]):
            scenario=category
            bottom=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
                axs[i].bar(index[j], costs[scenario][component] , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3,  )  #hatch=hatches[j], edgecolor='white'
                bottom+=costs[scenario][component]

            xlabels = ['base', 'default rooftop potential','high rooftop potential']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            #if i==1: 
            #    axs[i].set_xticklabels(xlabels,fontsize=16, rotation=0,ha='center', rotation_mode='anchor')
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25) 
            if i==0 : axs[i].set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            if j==0 : axs[i].set_ylabel(scenarios_description[scen_sector_c][0]+'\n'+
                scenarios_description[scen_sector_c][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')

             
            

for i ,scen in enumerate (scenario_format.keys()):
         fig.text(0.3, 0.365*(2-i)-0.02, "- "+str(round(scen_costs[scenario_format[scen][0]].sum()-
                                              scen_costs[scenario_format[scen][1]].sum(),2))
         +" bn€\n "+str(round(100*((scen_costs[scenario_format[scen][1]].sum()/
                                     scen_costs[scenario_format[scen][0]].sum())-1),2))+"%", 
         fontsize=18)
         fig.text(0.6, 0.326*(2-i)-0.02, "- "+str(round(scen_costs[scenario_format[scen][0]].sum()-
                                              scen_costs[scenario_format[scen][2]].sum(),2))
         +" bn€\n "+str(round(100*((scen_costs[scenario_format[scen][2]].sum()/
                                     scen_costs[scenario_format[scen][0]].sum())-1),2))+"%", 
         fontsize=18)
            
#line = lines.Line2D([0.3, 1], [0.3, 2],
#                    lw=2, color='black', axes=axs[0])



            
def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


arrow_style = dict(arrowstyle="simple", mutation_scale=15, ec=None)
add_arrow(axs[0], axs[0], (0.22, 0.5), (0.4, 0.5), fc="black", **arrow_style)
add_arrow(axs[0], axs[0], (0.605, 0.3), (0.78, 0.3), fc="black", **arrow_style)

add_arrow(axs[1], axs[1], (0.21, 0.5), (0.4, 0.5), fc="black", **arrow_style)
add_arrow(axs[1], axs[1], (0.6, 0.5), (0.8, 0.5), fc="black", **arrow_style)

axs[0].set_yticks(np.arange(0, 851, 170),fontsize=25)
axs[1].set_yticks(np.arange(0, 851, 170),fontsize=25)

axs[0].axhline(y=255, xmin=0.22, xmax=0.37, linewidth=4, color = 'k')
axs[0].axhline(y=255, xmin=0.38, xmax=0.6, linewidth=4,linestyle='--' ,color = 'k',dashes=(1, 1))



In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(4, 1, figsize=(3,22)) #, sharey=True)
fig.subplots_adjust(hspace=0.1)
fig.subplots_adjust(wspace=1)

bar_width=0.3
index=np.arange(4) 

scenarios_description=['Scenario A', 'Scenario B', 'Scenario C', 'Scenario D']

#main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['-nohomegen',''],
#          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
#          'sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],
#        'Sensitivity-runs/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],
#             }  #
main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['','-nohomegen'],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],
          }

for i,base in enumerate(main_scen):
            scenario=base.format(main_scen[base][0]) 
            scenario_with=base.format(main_scen[base][1]) 
            bottom_p=0
            bottom_n=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
              difference= costs[scenario_with][component]-costs[scenario][component]
              if difference > 0 :
                axs[i].bar(index[0], difference , bar_width, 
                           bottom=bottom_p,
                           color=tech_colors[component], zorder=3)
                bottom_p+=difference
              else:
                 axs[i].bar(index[0], difference , bar_width, 
                           bottom=bottom_n,
                           color=tech_colors[component], zorder=3)
                 bottom_n+=difference
                
            axs[i].axhline(y=0,lw=1.5, color='black',zorder=4)
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            axs[i].set_ylabel(scenarios_description[i]+
                              '\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 2.5),ncol=1,
           fontsize=20,frameon=False)


#axs[0].set_yticks(np.arange(0, 301, 50),fontsize=25)
axs[1].set_yticks(np.arange(-30, 31, 10),fontsize=25)
axs[2].set_yticks(np.arange(-40, 26, 10),fontsize=25)
axs[3].set_yticks(np.arange(-125, 101, 25),fontsize=25)


In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000
plt.rc('hatch', linewidth=1.5)



fig, axs = plt.subplots(3, 1, figsize=(5,18)) #, sharey=True)
fig.subplots_adjust(hspace=0.2)
fig.subplots_adjust(wspace=1)

bar_width=0.3
index=np.arange(4) 

scenarios_description=['Scenario A', 'Scenario B', 'Scenario C', 'Scenario D']

hatches=['','','','///']
#main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['-nohomegen',''],
#          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
#          'sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],}  #

for j,base in enumerate(main_scen):
            scenario=base.format(main_scen[base][0]) 
            scenario_with=base.format(main_scen[base][1]) 
            #axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            #axs[i].grid(axis = 'x',color='gainsboro', zorder=0)
            
            if j==3: 
                i=j-1
            else:
                i=j 
            for color, component in enumerate(costs[scenario].index):
               axs[i].scatter(costs[scenario][component], costs[scenario_with][component], 
                           s=6*max(costs[scenario][component],costs[scenario_with][component]),
                           c=tech_colors[component], label=component,
                              linewidths=1, alpha=0.6 ,hatch=hatches[j], edgecolors=tech_colors[component], zorder=3)
               axs[i].scatter(costs[scenario][component], costs[scenario_with][component], 
                           s=6*max(costs[scenario][component],costs[scenario_with][component]),
                           c='none', label=component,
                              linewidths=1,hatch=hatches[j], edgecolors=tech_colors[component], alpha=1, zorder=4)
                
            axs[i].grid(True, linestyle='--',color='lightgrey', which='major', zorder=0)    
            axs[i].axline([0, 0], [1, 1],lw=1.5, color='grey', linestyle='--', zorder=0)
            #axs[i].set_xlabel("", rotation=0)
            #axs[i].set_xticks([], [])
            #axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            if j!=3: 
                axs[i].set_ylabel(scenarios_description[i]+'\n'+
                scenarios_description[i]+'\n\n'+'with distributed techs. [bn€/a]',fontsize=20, fontname = 'Ubuntu')
            axs[i].set_xlabel('without distributed techs. [bn€/a]',fontsize=20, fontname = 'Ubuntu')
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))

legend1=plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 2.5),ncol=1,
           fontsize=20,frameon=False)
plt.gca().add_artist(legend1)

hatches_=['','///']

legend_manual_2=[]
for i, label in enumerate(['default rooftop potential', 'high rooftop potential']):
    legend_manual_2.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches_[i], edgecolor='k', 
                                        label=label))
legend2=plt.legend(handles=legend_manual_2,loc='center right', bbox_to_anchor=(2.2, 2.7),ncol=1,
           fontsize=20, frameon=False) 
plt.gca().add_artist(legend2)

axs[0].set_yticks(np.arange(0, 81, 20),fontsize=25)
axs[0].set_xticks(np.arange(0, 81, 20),fontsize=25)

axs[1].set_yticks(np.arange(0,81, 20),fontsize=25)
axs[1].set_xticks(np.arange(0,81, 20),fontsize=25)

axs[2].set_yticks(np.arange(0,181, 60),fontsize=25)
axs[2].set_xticks(np.arange(0,181, 60),fontsize=25)


In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(4, 1, figsize=(6,22)) #, sharey=True)
fig.subplots_adjust(hspace=0.1)
fig.subplots_adjust(wspace=0.5)

bar_width=0.4
index=np.arange(4) 

results1=results.drop(labels =['co2 vent', 'Total-cost [bn€]', 'oil','gas','coal','lignite'
                              , 'Peak load [GW]','DC','uranium','Max rooftop'])
results1 = results1.rename(index={ 'solar':'Solar utility'})
                                 
main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['','-nohomegen'],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],
          }
scenarios_description=['Scenario A', 'Scenario B', 'Scenario C', 'Scenario D']

for i,base in enumerate(main_scen):
    for j, category in enumerate(main_scen[base]):
            scenario=base.format(category) 
            bottom=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(results1[scenario].index):
                axs[i].bar(index[j], results1[scenario][component]/1e6 , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=results1[scenario][component]/1e6
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            if i==0 : axs[i].set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            if j==0 : axs[i].set_ylabel(scenarios_description[i]+
                                        '\n'+'Installed capacity [TW]',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (results1[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 2.5),ncol=1,
           fontsize=20,frameon=False)



axs[0].set_yticks(np.arange(0, 4.1, 1),fontsize=25)
axs[1].set_yticks(np.arange(0, 4.1, 1),fontsize=25)
axs[2].set_yticks(np.arange(0, 10.1, 2.5),fontsize=25)
axs[3].set_yticks(np.arange(0, 10.1, 2.5),fontsize=25)




In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(2, 1, figsize=(6,11)) #, sharey=True)
fig.subplots_adjust(hspace=0.1)
fig.subplots_adjust(wspace=0.5)

bar_width=0.4
index=np.arange(4) 

                                 
main_scen={
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':['','-nohomegen'],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':['-nohomegen',''],
          }
scenarios_description=['Scenario C', 'Scenario D']

"""
results=pd.DataFrame({})

for scen in main_scen.keys():
    for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen.format(main_scen[scen][i])
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                 ,pd.Series({'Heat store [Gwh]' : n.stores.e_nom_opt.filter(like='water').sum()/1e3})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
"""
           

results1=results.drop(labels =['co2 vent', 'Total-cost [bn€]', 'oil','gas','coal','lignite'
                              , 'Peak load [GW]','DC','uranium','process emissions','process emissions CC',
                               'gas for industry','gas for industry CC','solid biomass for industry',
                               'solid biomass for industry CC','V2G', 'BEV charger'
                              ])

for ix in results1.index :   
                if 'resistive heater' in ix :results1 =results1.rename(index={ix:'resistive heater',})
                if 'heat pump' in ix :results1 =results1.rename(index={ix:'heat pump',})
                if 'gas boiler' in ix :results1 =results1.rename(index={ix:'gas boiler',})
                if 'gas CHP' in ix :results1 =results1.rename(index={ix:'gas CHP',})    
                if 'biomass' in ix :results1 =results1.rename(index={ix:'biomass',})
                if 'solar thermal' in ix :results1 =results1.rename(index={ix:'solar thermal',})
                if 'water tank' in ix :results1 =results1.rename(index={ix:'Heat storage',})
                if 'biogas' in ix :results1 =results1.rename(index={ix:'Biogas',})

                    
results1 = results1.rename(index={ 'solar':'solar utility', 'solar rooftop': 'solar distributed'})

                            
results1=results1.groupby(results1.index).sum() 
#results1.loc['Heat storage']=results1.loc['Heat store [Gwh]']   ###e_nom_opt
results1=results1.drop(labels =['Heat store [Gwh]','Heat storage','Biogas'])

for i,base in enumerate(main_scen):
    for j, category in enumerate(main_scen[base]):
            scenario=base.format(category) 
            bottom=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(results1[scenario].index):
                axs[i].bar(index[j], results1[scenario][component]/1e6 , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=results1[scenario][component]/1e6
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            if i==0 : axs[i].set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            if j==0 : axs[i].set_ylabel(scenarios_description[i]+
                                        '\n'+'Installed capacity [TW]',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (results1[scenario].index):
  if  key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1.2, 2.5),ncol=1,
           fontsize=20,frameon=False)






In [ ]:
import re

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 1
plt.rcParams['text.color'] = 'black'

#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(3, 1, figsize=(8,15))
fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=1.5)

bar_width=0.3
index=np.arange(5) 


main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1{}-higheff_2030.nc':['-nohomegen',''],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],
          '{}/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}{}_2030.nc':
           [['Stable-sector-coupled','-nohomegen',''],
            ['Stable-sector-coupled','',''],
            ['Stable-sector-coupled','','-highrooftop6']],
          }

components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['','\\\\','xx', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']



for i,base in enumerate(main_scen):
    for j, category in enumerate(main_scen[base]):
            if type(category)==list : scenario=base.format(*category)
            else : scenario=base.format(category)   
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs[i].bar(index[k]+j*bar_width, results[scenario][component]/1e3 , bar_width,
                           color=tech_colors[component],label=component, edgecolor='white', hatch=hatches[j], zorder=3)
                if component=='solar rooftop':
                     axs[i].bar(index[k]+j*bar_width, results[scenario]['Max rooftop']/1e3 , bar_width,
                           fill = False, edgecolor='chocolate', zorder=2)
            axs[i].axhline(y=results[scenario]['Peak load [GW]'],lw=1.5, color='darkblue',zorder=4)
            #axs[i].hlines(y=results[scenario]['Max rooftop'].sum()/1e3,
             #              xmin=index[2]+0.5*bar_width, xmax=index[2]+1.5*bar_width ,
              #             lw=1.7, color='chocolate',zorder=4)            
            axs[i].set_ylabel(scenarios_description[scenario][0]
                             +"\n\ninstalled capacity [GW]", rotation=90,fontsize=18 )  
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
    axs[i].set_title(scenarios_description[scenario][2]+'\n',fontsize=20, fontname = 'Ubuntu')
    axs[i].set_xticks((index)++bar_width*0.5)
    xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Distributed battery', 'Distribution grid']
    xlabels = [label.replace(' ', ' \n') for label in xlabels]
    axs[i].set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
    if j==0 : axs[i].set_ylabel(scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')

axs[0].set_yticks(np.arange(0, 1501, 250),fontsize=20)
axs[1].set_yticks(np.arange(0, 1501, 250),fontsize=20)
axs[2].set_yticks(np.arange(0, 4201, 1000),fontsize=20)
        
legend_manual=[]
for i, label in enumerate(['without distributed generation', 'with distributed generation','with distributed generation,\nhigh DPV assumption']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', 
                                        label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.8, 4.7),ncol=1,
           fontsize=18, frameon=False) 
plt.gca().add_artist(legend1)

lines = plt.gca().get_lines()
legend2 = plt.legend([lines[i] for i in np.arange(2)], (['Peak low voltage electricity\ndemand', 'Max. distributed solar capacity' ])
                     ,bbox_to_anchor=(1.79,4.4), fontsize=18, frameon=False)
plt.gca().add_artist(legend2)

plt.savefig('figure 4.pdf', dpi=1000)

In [ ]:
components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']

for i,base_ in enumerate(main_scen):
    scenario=base_.format(main_scen[base_][0])
    base=base_.format('')
    print(base +': \n')
    print(scenario)
    
    for k, component in enumerate(components):
        reduction =((results[scenario][component]
                             -results[base][component])*100/results[scenario][component])
        print(component+' reduction: ',str(reduction)+ '%')
    print('solar rooftop share: ',str(100*results[scenario]['solar rooftop']/504770.5336446769), 
                                     str(100*results[base]['solar rooftop']/504770.5336446769) )
                                      
    print('solar utility share: ',str(100*results[scenario]['solar']/10560308.087845214), 
                                     str(100*results[base]['solar']/10560308.087845214)  )
    
    print('home batteries to peak demand',  str(0.1*results[base]['home battery discharger']/results[base]['Peak load [GW]'])) 
    print('batteries to peak demand',  str(0.1*results[scenario]['battery discharger']/results[scenario]['Peak load [GW]'])
                 ,str(0.1*results[base]['battery discharger']/results[base]['Peak load [GW]'])) 
    print('dist. grid to peak demand',  str(0.1*results[scenario]['electricity distribution grid']/results[scenario]['Peak load [GW]'])
                 ,str(0.1*results[base]['electricity distribution grid']/results[base]['Peak load [GW]'])) 

In [ ]:
main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc'   #no h2 network for all files in folder
           :['With distributed generation', 'scenario A'],
           'Sensitivity-runs/postnetworks/elec_s_181_lv1.5__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1_2030.nc'
           :['With distributed generation', 'scenario A'],
           'Sensitivity-runs/postnetworks/elec_s_181_lvopt__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1_2030.nc'
           :['With distributed generation', 'scenario A'],
           'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc'
           :['With distributed generation', 'scenario B'],
           'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.5__Co2L0p05-2H-solar+p3-dist2_2030.nc'
           :['With distributed generation', 'scenario B'],
           'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lvopt__Co2L0p05-2H-solar+p3-dist2_2030.nc'
           :['With distributed generation', 'scenario B'],}
         
          #'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv{}__Co2L0p05-2H-solar+p3-dist2_{}.nc'
          # :['With distributed generation', 'scenario C'],
          #          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv{}__Co2L0p05-2H-solar+p3-dist2_{}.nc'
          # :['With distributed generation', 'scenario C'],
          #          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv{}__Co2L0p05-2H-solar+p3-dist2_{}.nc'
          # :['With distributed generation', 'scenario C'],}





results_tr=pd.DataFrame({})
results_tr_p=pd.DataFrame({})
transmission_expansion=[1.1, 1.5, 'opt']
year=2030
for scenario in main_scen.keys():
  #for tr_ex in transmission_expansion: 
             #print(scenario.format(tr_ex,year))
             result_p=pd.DataFrame({})
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
                
             for carrier in ['solar','battery discharger','solar rooftop',    # for power generation
                  'home battery discharger', 'electricity distribution grid','onwind'] :  
               if  n.generators_t.p[[i for i in n.generators.index if n.generators.carrier[i]==carrier]].sum().sum():  
                 result_p=pd.concat([result_p, pd.Series({carrier : 
                                                          2*n.generators_t.p[[i for i in n.generators.index if n.generators.carrier[i]==carrier]].sum().sum()
                                                         })])
               if  n.links_t.p0[[i for i in n.links.index if n.links.carrier[i]==carrier]].sum().sum():  
                 result_p=pd.concat([result_p, pd.Series({carrier : 
                                                          2*n.links_t.p0[[i for i in n.links.index if n.links.carrier[i]==carrier]].sum().sum()
                                                         })])
             results_tr_p[scenario]=result_p
            
             results_tr[scenario]=result.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                                   'gas','uranium','coal' ,'lignite','oil',
                                                   'DC'])  # 'ror','OCGT','H2 Electrolysis' ,H2 pipeline

In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 1
#plt.rcParams['figure.dpi'] = 1000

fig, axs = plt.subplots(2,1, figsize=(10,15))
fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=1.5)

bar_width=0.3
index=np.arange(6) 

components=['solar','battery discharger','solar rooftop','home battery discharger',
            'electricity distribution grid','onwind',]
hatches = ['','////','\\\\','..','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

counter=0

#for j,tr_ex in enumerate(transmission_expansion):
for i in range(2):
 for j in range (3):   
  scenario = (list(main_scen.keys()))[counter] 
  print(i, scenario)
  for k, component in enumerate(components): 
    axs[i].bar(index[k]+j*bar_width, results_tr[scenario][component]/1e3 , bar_width,    #1e3 : MW to GW
          color=tech_colors[component],label=component, edgecolor='white', hatch=hatches[j], zorder=3)
  axs[i].axhline(y=n.loads_t.p.sum(axis=1).max()/1e3,lw=1.5, color='darkblue',zorder=4)
  axs[i].axhline(y=n.generators.p_nom_max.filter(like='solar rooftop').sum()/1e3,lw=1.5, color=tech_colors['solar rooftop'],zorder=4)            
  axs[i].set_ylabel(main_scen[scenario][1]+"\ninstalled capacity [GW]\n", rotation=90,fontsize=18 )  
  axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
  axs[i].set_yticks(np.arange(0, 1001, 200),fontsize=16)
  #axs[i].set_title(scenarios_description[scenario][2]+'\n',fontsize=20, fontname = 'Ubuntu')
  axs[i].set_xticks((index)++bar_width*0.5)
  xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid', 'Onshore wind']
  xlabels = [label.replace(' ', ' \n') for label in xlabels]
  axs[i].set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
  #if j==0 : axs[i].set_ylabel(scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
  axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
  counter+=1


legend_manual=[]
for i, label in enumerate(transmission_expansion):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=str(label)+' transmission expansion'))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.5, 0.6),ncol=1, #title='Transmission',
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)

lines = plt.gca().get_lines()
legend2 = plt.legend([lines[i] for i in np.arange(2)], (['Peak low voltage demand','Max. dist. solar capacity', ])
                     ,bbox_to_anchor=(1.5,0.5), fontsize=16, frameon=False)
plt.gca().add_artist(legend2)


In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 1
#plt.rcParams['figure.dpi'] = 1000

fig, axs = plt.subplots( 2,1, figsize=(10,14))
fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=1.5)

bar_width=0.3
index=np.arange(len(results_tr_p.index)) 

components=['solar','battery discharger','solar rooftop','home battery discharger',
            'electricity distribution grid','onwind',]
hatches = ['','////','\\\\','..','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

counter=0
#for j,tr_ex in enumerate(transmission_expansion):
for i in range(2):
 for j in range (3):   
  scenario = (list(main_scen.keys()))[counter] 
  print(i , scenario)
  for k, component in enumerate(results_tr_p.index): 
    axs[i].bar(index[k]+j*bar_width, results_tr_p[scenario][component]/1e6 , bar_width,      #TWh  #scenario.format(tr_ex,year)
          color=tech_colors[component],label=component, edgecolor='white', hatch=hatches[j], zorder=3)
  #axs[i].axhline(y=n.loads_t.p.sum(axis=1).max()/1e3,lw=1.5, color='darkblue',zorder=4)
  #axs[i].axhline(y=n.generators.p_nom_max.filter(like='solar rooftop').sum()/1e3,lw=1.5, color=tech_colors['solar rooftop'],zorder=4)            
  axs[i].set_ylabel(main_scen[scenario][1]+
                    "\nEnergy generation/transmission [TWh]\n", rotation=90,fontsize=18 )  
  axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
  #axs[i].set_yticks(np.arange(0, 1001, 200),fontsize=16)
  #axs[i].set_title(scenarios_description[scenario][2]+'\n',fontsize=20, fontname = 'Ubuntu')
  axs[i].set_xticks((index)++bar_width*0.5)
  xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid', 'Onshore wind']
  xlabels = [label.replace(' ', ' \n') for label in xlabels]
  axs[i].set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
  #if j==0 : axs[i].set_ylabel(scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
  axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
  counter+=1  
            
legend_manual=[]
for i, label in enumerate(transmission_expansion):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=str(label)+' transmission expansion'))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.5, 0.6),ncol=1, #title='Transmission',
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)



In [ ]:
import pypsa
sens = {}
for dist_factor in [0,1,2,5]:
    for efficiency in [1, 0.9, 0.75]:
      if dist_factor==1 and efficiency ==1 :
        case= ('electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc')
      elif dist_factor==2 and efficiency ==0.9 :
        case= ('electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc')
      else:  
         case= ('Sensitivity-runs/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist{}-noH2network-higheff{}_2030.nc'.
                         format(str(dist_factor), str(efficiency)))
      n=pypsa.Network(case)
      sens['dist:'+str(dist_factor)+', eff:'+str(efficiency)] = n.generators.p_nom_opt.filter(like='solar rooftop').sum() 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mc

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['figure.dpi'] = 100

xlist = np.array([0,1,2,5])
ylist = np.array([0,10,25])
X, Y = np.meshgrid(xlist, ylist)
Z = np.array([[0.47728170205762266 , 8045.285351595507 ,92469.98349142022,    317795.7771892865],
           [3654.6472951433398 ,78086.19163099737,      307408.3379410844,   442321.13514019124],
           [435484.7168591219 ,470206.12890221796,      482330.60048385686,   496376.37876425066] ])

#fig,ax=plt.subplots()
#cp = ax.contourf(X, Y, Z/1e3 , cmap ='viridis_r')  #,400 
levels=list(sens.values())  #[0,3654,8045,78086,317795,435484,442321,470206,496376]
levels.sort()
levels1 = [round(x/1e3,1) for x in levels]


cp = plt.contourf(X, Y, Z/1e3, cmap = 'viridis_r',levels=levels1)
cbar=plt.colorbar(cp,)

"""
levels1=(cp.levels)  #has to run before specifying 400 as levels first

norm= matplotlib.colors.Normalize(vmin=cp.cvalues.min(), vmax=cp.cvalues.max())
sm = plt.cm.ScalarMappable(norm=norm, cmap = cp.cmap)
sm.set_array([])
cbar= fig.colorbar(sm, ticks=levels1)

cmap = matplotlib.colors.ListedColormap(['royalblue', 'cyan',
                                  'yellow', 'orange','b','white','navy','red','purple','pink'])

norm = matplotlib.colors.BoundaryNorm(levels,  cmap.N)
cbar = matplotlib.colorbar.ColorbarBase(ax, cmap= cmap,
                                norm=norm,
                                boundaries=[0] + levels + [13],
                                extend='both',
                                ticks=levels,
                                spacing='proportional',
                                orientation='horizontal')
"""
#cbar= fig.colorbar(cp, ticks=levels) # Add a colorbar to a plot
cbar.set_ticks(levels1)
cbar.set_ticklabels(levels1)
#cbar.tick_params(labelsize=14)
#cbar.ax.set_xticks(np.linspace(0, 150, 1))
#cbar.ax.set_xticklabels(levels1)
cbar.set_label('Distributed PV installed capacity [GW]',size=14,labelpad=8)

plt.xlabel('Distribution grid\n  cost factor relative to base cost (500€/kW)', fontsize=14)
plt.ylabel('Distribution Grid power loss [%]' ,fontsize=14)
plt.axhline(y=10, color='white', linestyle='--')
plt.axhline(y=0.01, color='white', linestyle='--')
plt.axhline(y=24.95, color='white', linestyle='--')

plt.axvline(x=1, color='white', linestyle='--')
plt.axvline(x=0.01, color='white', linestyle='--')
plt.axvline(x=2, color='white', linestyle='--')
plt.axvline(x=4.99, color='white', linestyle='--')

plt.show()

In [ ]:
#scenario A:

scenarios_tr=[
    'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc',
    'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc',
'Sensitivity-runs/postnetworks/elec_s_181_lv1.5__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1-nohomegen_2030.nc',
'Sensitivity-runs/postnetworks/elec_s_181_lv1.5__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1_2030.nc',
'Sensitivity-runs/postnetworks/elec_s_181_lvopt__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1-nohomegen_2030.nc',
'Sensitivity-runs/postnetworks/elec_s_181_lvopt__Co2L0p05-2H-solar+p3-dist1-noH2network-higheff1_2030.nc',]
costs=pd.DataFrame({})
results_tr=pd.DataFrame({})

for scen in scenarios_tr:
             scenari =  scen
             n=pypsa.Network(scenari)
             scen_costs = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().sum()/(1e9)  #in billiion Euros
             results_tr[scen]=pd.Series({'Total-cost' : scen_costs}) #billion euros

In [ ]:
#scenario B:
import pypsa
import openpyxl
import pandas as pd
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

Dist_factor =[2] 
transmission_expansion= [1.1, 1.5, 'opt'] 
years = [2030] 
results_tr=pd.DataFrame({})
scenarios_tr={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv{}__Co2L0p05-2H-solar+p3-dist2{}_2030.nc':['-nohomegen',''],}
scen_costs={}
costs=pd.DataFrame({})

for scen in scenarios_tr.keys():
   for tr_ex in transmission_expansion: 
     for option in ['','-nohomegen']:
             scenari =  scen.format(tr_ex,option)
             n=pypsa.Network(scenari)
             scen_costs = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().sum()/(1e9)  #in billiion Euros
             results_tr['tr: '+str(tr_ex)+option]=pd.Series({'Total-cost' : scen_costs}) #billion euros
             
#results.to_excel("output_tr-181-newbase.xlsx")  
results_tr

In [ ]:
#battery sensitivity
import pypsa
import openpyxl
import pandas as pd
import logging
#logger.setLevel(logging.INFO)


main_scen=['electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc',
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2-nohomebattery_2030.nc']

results_battery=pd.DataFrame({})


for scenario in main_scen:
 #for tr_ex in transmission_expansion: 
   #  for year in years:
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
             results_battery[scenario]=result.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                                   'gas','uranium','coal' ,'lignite','oil',
                                                   'DC'])  #,'H2 pipeline'



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 1
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j, scenario in enumerate(main_scen):
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results_battery[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("Scenario B\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            axs.set_yticks(np.arange(0, 1401, 200),fontsize=20)
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['with\nhome battery', 'without\nhome battery']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.35, 0.5),ncol=1,
           fontsize=18, frameon=False) 
plt.gca().add_artist(legend1)

In [ ]:
#battery sensitivity
import pypsa
import openpyxl
import pandas as pd
import logging
#logger.setLevel(logging.INFO)


main_scen=['sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc',
          'sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-noEVbattery-nohomebattery_2030.nc',
          'Sensitivity-runs/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-noEVbattery-nohomebattery-notes_2030.nc']

results_battery=pd.DataFrame({})


for scenario in main_scen:
 #for tr_ex in transmission_expansion: 
   #  for year in years:
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
             results_battery[scenario]=result.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                                   'gas','uranium','coal' ,'lignite','oil',
                                                   'DC'])  #,'H2 pipeline'



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 0.2
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['..','xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j,base in enumerate(main_scen):
            scenario=base
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results_battery[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("scenario C\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Distributed battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['with home battery',
                           'without home battery\nand EV batteries', 
                           'without home battery\nand EV and tes']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.6, 0.5),ncol=1,
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)

In [ ]:
#sensitivity for cases WITHOUT losses

import pypsa
import openpyxl
import pandas as pd
import logging
#logger.setLevel(logging.INFO)

main_scen={'electricity_sector_only_181_newbase':'postnetworks/elec_s_181_lv1.0__Co2L0-2H-solar+p3-dist2_2030.nc',
    'electricity_sector_only_181_new':'postnetworks/elec_s_181_lv1.0__Co2L0-2H-solar+p3-dist2_2030.nc',
    'electricity_sector_only_181_newbase_norooftop':'postnetworks/elec_s_181_lv1.0__Co2L0-2H-solar+p3-dist2_2030.nc'}
### the second network has batteries, but no rooftop

results_test=pd.DataFrame({})
transmission_expansion= [1.0]
years=[2030]

for scenario in main_scen.keys():
 for tr_ex in transmission_expansion: 
    for year in years:
             n=pypsa.Network(scenario+'/'+main_scen[scenario].format(tr_ex,year))
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
             results_test[scenario]=result.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                                   'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                                   'DC']) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches


plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 0.2
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['..','xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j,base in enumerate(main_scen.keys()):
            scenario=base
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results_test[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("scenario B\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            axs.set_yticks(np.arange(0, 1201, 200),fontsize=20)
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['with rooftop and battery', 'without rooftop, with battery', 
                          'with neither rooftop nor battery']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.4, 0.5),ncol=1,
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)

# sensitivity of scenario C and D to storage

In [ ]:
#scenrio C
main_scen={
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':['-nohomegen',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-noEVbattery-nohomebattery_2030.nc':['Scenario C :no (BEVs+home batteries)',''],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-noEVbattery-nohomebattery-notes_2030.nc':['Scenario C :no (BEVs+home batteries+tes)','']
}


In [ ]:
#with highrooftop

main_scen={
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6_2030.nc':['Scenario C highrooftop',''], 
         'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6-noEVbattery-nohomebattery_2030.nc':['Scenario C highrooftop:no (BEVs+home batteries)',''],
       'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6-noEVbattery-nohomebattery-notes_2030.nc':['Scenario C highrooftop:no (BEVs+home batteries+tes)','']
   }

In [ ]:



results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name

for scen in main_scen.keys():
    #for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
             scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
             scen_costs[scenario] = scen_costs[scenario].rename(index={
                          'battery':'Battery storage','battery charger':'Battery storage','battery discharger':'Battery storage',
                          'home battery':'Home battery storage','home battery charger':'Home battery storage','home battery discharger':'Home battery storage',                                                       
                           'offwind-ac':'Offshore wind','offwind-dc':'Offshore wind', 'onwind':'Onshore wind',
                          'AC':'Transmission lines'  , 'DC':'Transmission lines','ror': 'Run of river',
                          'hydro':'Hydroelectricity' , 'nuclear':'Nuclear','uranium':'Nuclear',
                          'solar':'Solar utility', 'solar rooftop':'Solar rooftop',
                          'electricity distribution grid' : 'Electricity distribution grid',
                           'PHS' : 'Pumped hydro storage', 'H2':'H2 storage',
                           'SMR':'Power to gas'  , 'SMR CC':'Power to gas','oil': 'Oil and gas to power/heat',
                           'Fischer-Tropsch':'Power to liquid', 
                           'Sabatier':'Power to gas', 'helmeth':'Power to gas', 'coal' : 'Coal', 'lignite' :'Coal',
                          })  # co2 vent','co2','co2 stored',
             for ix in scen_costs[scenario].index :   
                if 'resistive heater' in ix or 'heat pump' in ix :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix : 'Power to heat',})
                if  any(w in ix for w in ['gas', 'OCGT', 'CCGT'])  :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Oil and gas to power/heat',})
                if 'biomass' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biomass',})
                if 'solar thermal' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Solar thermal',})
                if 'water tank' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Heat storage',})
                if 'biogas' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biogas',})
                #if 'biogas' in ix :scen_costs[scenario].rename(index={ix:'Biomass',})
                    
             scen_costs[scenario]=scen_costs[scenario].groupby(scen_costs[scenario].index).sum() 

In [ ]:
tech_colors= {
    'Transmission lines': "grey",
    'Battery storage' : '#80c944' ,
        'battery charger' : '#80c944' ,
    'battery discharger' : 'darkgreen' ,    
    'Home battery storage' : '#ace37f'  ,
        'home battery charger' : '#ace37f'  ,
    'home battery discharger' : '#ace37f' ,    
    'H2 Fuel Cell': '#c251ae',
    'H2 Electrolysis': '#ff29d9',
    'H2 pipeline' : 'plum',
    'SMR': '#870c71',
    'SMR CC': '#4f1745',
    'Sabatier' : '#4f1745',
    'helmeth' : '#4f1745',
    'co2':'linen',
    'co2 vent':'linen',
    'co2 stored':'linen',
    'Electricity distribution grid': 'darkgrey',
    'electricity distribution grid': 'darkgrey',
    'uranium': '#ff8c00',
    "onwind": "#235ebc",
    "Onshore wind": "#235ebc",
    "onshore wind": "#235ebc",
    'offwind': "#6895dd",
    'offwind-ac': "#6895dd",
    'Offshore wind': "#6895dd",
    'offshore wind': "#6895dd",
    'offshore wind ac': "#6895dd",
    'offwind-dc': "#74c6f2",
    'offshore wind dc': "#74c6f2",
    "Hydroelectricity": "#08ad97",
    "hydro": "#08ad97",
    "hydro+PHS": "#08ad97",
    "PHS": "#08ad97",
    "Pumped hydro storage": "teal",
    "hydro reservoir": "#08ad97",
    'hydroelectricity': '#08ad97',
    "ror": "#4adbc8",
    "Run of river": "#4adbc8",
    'Solar': "#f9d002",
    'Solar utility': "#f9d002",
    'solar': "#f9d002",
    'solar PV': "#f9d002",
    'Solar rooftop' : 'orange',
    'solar rooftop' : 'orange',
    'solar thermal': '#ffef60',
    'biomass': 'olivedrab', #'#0c6013',
    'solid biomass': '#06540d',
    'biogas': '#23932d',
    'waste': '#68896b',
    'geothermal': '#ba91b1',
    "OCGT": "#d35050",
    "gas": "#d35050",
        "CCGT": "#d35050",
    "natural gas": "#d35050",
    "CCGT": "#b20101",
    "Nuclear": "rosybrown",
        "nuclear": "rosybrown",
    "coal": "#707070",
    "lignite": "#9e5a01",
    "oil": "#262626",
    "H2 storage": "#ea048a",
    "hydrogen storage": "#ea048a",
    "battery": "#b8ea04",
    "Electric load": "#f9d002",
    "electricity": "#f9d002",
    "lines": "#70af1d",
    "transmission lines": "#70af1d",
    "AC-AC": "#70af1d",
    "AC line": "#70af1d",
    "links": "#8a1caf",
    "HVDC links": "#8a1caf",
    "DC-DC": "#8a1caf",
    "DC link": "#8a1caf",
    'Solar thermal': '#ffef60',
    "Oil and gas to power/heat":"#d35050",
    "Coal": "#707070",
    "Power to liquid": '#4f1745',
    "Power to gas":'#870c71',
    "Power to heat":'maroon',
    "Biomass":'#06540d',
    "Biogas":'#23932d',
    "BEV charger":'rebeccapurple',
    "V2G":'mediumpurple',
    "DAC":'lightcoral',
    "Heat storage":"brown",
    "heat pump":'darkslategrey',
    "resistive heater":'maroon',
    "Fischer-Tropsch":"grey",
    'biogas to gas':'greenyellow',
    'industry' : 'black',
    'industry gas' : 'darkred',
    'industry biomass' : 'olive',


}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches


plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 0.2
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['..','xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j,base in enumerate(main_scen.keys()):
            scenario=base
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("scenario C\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            axs.set_yticks(np.arange(0, 4201, 1000),fontsize=20)
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['with distributed storage techs.',
                           'with neither home battery\nnor EV batteries', 
                           'with neither home battery\nnor EV batteries,\nnor thermal storage',]):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.55, 0.5),ncol=1,
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)

# with highrooftop


In [ ]:
results.to_csv('tables/results_hr.csv')

In [ ]:
results.loc['solar rooftop']

In [ ]:
3.507501e+05/2.170393e+06

In [ ]:
4.949653e5/2.170393e+06

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches


plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 0.2
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['..','xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j,base in enumerate(main_scen.keys()):
            scenario=base
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("scenario D\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            axs.set_yticks(np.arange(0, 4201, 1000),fontsize=20)
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Home battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['with distributed storage techs.',
                           'with neither home battery\nnor EV batteries', 
                           'with neither home battery\nnor EV batteries,\nnor thermal storage',]): 
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.55, 0.5),ncol=1,
           fontsize=16, frameon=False) 
plt.gca().add_artist(legend1)

### sector coupled sens : dist cost and losses effect on heat generation mix

In [ ]:
results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name



main_scen=[
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist1-noH2network-higheff1_2030.nc',
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc']

#          'Sensitivity-runs/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist1-noH2network-higheff0.93_2030.nc',

for scen in main_scen:
    #for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
             scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
             scen_costs[scenario] = scen_costs[scenario].rename(index={
                          'battery':'Battery storage','battery charger':'Battery storage','battery discharger':'Battery storage',
                          'home battery':'Home battery storage','home battery charger':'Home battery storage','home battery discharger':'Home battery storage',                                                       
                           'offwind-ac':'Offshore wind','offwind-dc':'Offshore wind', 'onwind':'Onshore wind',
                          'AC':'Transmission lines'  , 'DC':'Transmission lines','ror': 'Run of river',
                          'hydro':'Hydroelectricity' , 'nuclear':'Nuclear','uranium':'Nuclear',
                          'solar':'Solar utility', 'solar rooftop':'Solar rooftop',
                          'electricity distribution grid' : 'Electricity distribution grid',
                           'PHS' : 'Pumped hydro storage', 'H2':'H2 storage',
                           'SMR':'Power to gas'  , 'SMR CC':'Power to gas','oil': 'Oil and gas to power/heat',
                           'Fischer-Tropsch':'Power to liquid', 
                           'Sabatier':'Power to gas', 'helmeth':'Power to gas', 'coal' : 'Coal', 'lignite' :'Coal',
                          })  # co2 vent','co2','co2 stored',
             for ix in scen_costs[scenario].index :   
                if 'resistive heater' in ix or 'heat pump' in ix :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix : 'Power to heat',})
                if  any(w in ix for w in ['gas', 'OCGT', 'CCGT'])  :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Oil and gas to power/heat',})
                if 'biomass' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biomass',})
                if 'solar thermal' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Solar thermal',})
                if 'water tank' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Heat storage',})
                if 'biogas' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biogas',})
                #if 'biogas' in ix :scen_costs[scenario].rename(index={ix:'Biomass',})
                    
             scen_costs[scenario]=scen_costs[scenario].groupby(scen_costs[scenario].index).sum() 
            
            
costs=pd.DataFrame({})

preferred_order = pd.Index(
    [
        "Transmission lines",
        "Electricity distribution grid",
        "Oil and gas to power/heat",
        "Coal",
        "Power to liquid",
        "Power to gas",
        "Biomass",
        "Biogas",
        "BEV charger",
        "V2G",
        "DAC",
        "Power to heat",
        "Heat storage",
        "H2 storage",
        "H2 pipeline",
        "H2 Electrolysis",
        "H2 Fuel Cell",
        "Pumped hydro storage",
        "Hydroelectricity",
        "Run of river",
        "Nuclear",
        "Onshore wind",
        "Offshore wind",
        "Battery storage",
        "Home battery storage",
        "Solar thermal",
        "Solar utility",
        "Solar rooftop",
    ]
)


for  scenario in reversed(scen_costs.keys()) :
    costs[scenario]=pd.DataFrame(scen_costs[scenario].drop(labels =['co2 vent','co2','co2 stored'
                                                                   ]))
                                                                  #  ,'oil','uranium','lignite','coal','gas' ]) ) 
                                                        #   ['SMR', 'SMR CC','Sabatier','co2 vent','co2','co2 stored',
                                                   #    'helmeth', 'oil','uranium','lignite','coal','gas', 'CCGT','OCGT'
                                                  #  ]) ) #dropping all the variables not visible in figure
    

order = preferred_order.intersection(costs[scenario].index).append(
        costs[scenario].index.difference(preferred_order))    
costs=costs.reindex(order)   
costs=costs.drop(labels =['process emissions','process emissions CC','Li ion'])
costs=costs.fillna(0)
results=results.fillna(0)
            

In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000

fig, axs = plt.subplots( figsize=(10,10)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

bar_width=0.3
index=np.arange(4) 


for j, category in enumerate(main_scen):
            scenario=category
            bottom=0
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
                axs.bar(index[j], costs[scenario][component] , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=costs[scenario][component]
            axs.set_xlabel("", rotation=0)
            axs.set_xticks([], [])
            axs.tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            #if i==0 : axs[i].set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            #if j==0 : axs[i].set_ylabel(scenarios_description[scenario][0]+'\n'+
             #   scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
            axs.set_title('Scenario C\n 500 €/kW, 0 loss                      1000€/kW, 10%loss\n',   #0 loss        500 €/kW, 7% loss  
                          fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1,1),ncol=1,
           fontsize=16,frameon=False)

#fig.text(0.45, 0.25*(3-0), "- "+str(round(scen_costs[main_scen[0]].sum()-
 #                                             scen_costs[main_scen[1]].sum(),2))
  #       +" bn€\n "+str(round(100*((scen_costs[main_scen[1]].sum()/
   #                                  scen_costs[main_scen[0]].sum())-1),2))+"%", 
    #     fontsize=18)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['hatch.linewidth'] = 1
#plt.rcParams['figure.dpi'] = 1000


bar_width=0.3
index=np.arange(5) 

fig, axs = plt.subplots( figsize=(10,5))


components=['solar','battery discharger','solar rooftop','home battery discharger','electricity distribution grid']
hatches = ['','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']

for j, scenario in enumerate(main_scen):
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for k, component in enumerate(components):
                axs.bar(index[k]+j*bar_width, 
                           results[scenario][component]/1e3 , 
                           bar_width,
                           color=tech_colors[component],
                           label=component, 
                           edgecolor='white',
                           hatch=hatches[j],
                           zorder=3)
                
            axs.set_ylabel("Scenario C\ninstalled capacity [GW]\n", 
                              rotation=90,
                              fontsize=18 ) 
            
            axs.tick_params(axis='y',
                               labelsize=18)
            
            axs.set_yticks(np.arange(0, 4201, 1000),fontsize=20)
            
            xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Distributed battery', 'Distribution grid']
            xlabels = [label.replace(' ', ' \n') for label in xlabels]
            axs.set_xticks((index)++bar_width*0.5)    
            axs.set_xticklabels(xlabels,fontsize=16, rotation=60,ha='right', rotation_mode='anchor')
            
legend_manual=[]
for i, label in enumerate(['500 €/kW, 0 loss', '1000€/kW, 10%loss']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.45, 0.5),ncol=1,
           fontsize=18, frameon=False) 
plt.gca().add_artist(legend1)

In [ ]:
from pypsa.plot import projected_area_factor
from matplotlib.patches import Circle, Ellipse, Patch
from matplotlib.legend_handler import HandlerPatch
from matplotlib.lines import Line2D

class HandlerCircle(HandlerPatch):
    """
    Legend Handler used to create circles for legend entries.

    This handler resizes the circles in order to match the same dimensional
    scaling as in the applied axis.
    """

    def create_artists(
        self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans
    ):
        fig = legend.get_figure()
        ax = legend.axes

        unit = np.diff(ax.transData.transform([(0, 0), (1, 1)]), axis=0)[0][1]
        radius = orig_handle.get_radius() * unit * (72 / fig.dpi)
        center = 5 - xdescent, 3 - ydescent
        p = plt.Circle(center, radius)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]


def make_legend_circles_for(sizes, scale=1.0, **kw):
    return [Circle((0, 0), radius=(s / scale)**0.5, **kw) for s in sizes]

def add_legend_circles(
    ax, sizes, labels, scale=1, srid=None, patch_kw={}, legend_kw={}
):

    if srid is not None:
        area_correction = projected_area_factor(ax, n.srid) ** 2
        sizes = [s * area_correction for s in sizes]

    handles = make_legend_circles_for(sizes, scale, **patch_kw)

    legend = ax.legend(
        handles, labels, handler_map={Circle: HandlerCircle()}, **legend_kw
    )

    ax.add_artist(legend)


def add_legend_lines(ax, sizes, labels, scale=1, patch_kw={}, legend_kw={}):

    handles = [Line2D([0], [0], linewidth=s / scale, **patch_kw) for s in sizes]

    legend = ax.legend(handles, labels, **legend_kw)

    ax.add_artist(legend)


def add_legend_patch(ax, colors, labels, patch_kw={}, legend_kw={}):

    handles = [Patch(facecolor=c, **patch_kw) for c in colors]

    legend = ax.legend(handles, labels, **legend_kw)

    ax.add_artist(legend)

In [ ]:
#average net injection

import cartopy.crs as ccrs
import cartopy
import geopandas as gpd
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
epsg = 3035
nodes = gpd.read_file('electricity_sector_only_181_newbase_highloss_noH2network/regions_onshore_elec_s_181.geojson').set_index("name")
nodes["Area"] = nodes.to_crs(epsg=epsg).area.div(1e6)
    

fig,axs = plt.subplots(3,4, figsize=(9,9) , subplot_kw={"projection":ccrs.PlateCarree()})   #or  ccrs.Orthographic() .Mercator
fig.subplots_adjust(wspace=0.1)


carriers = {'solar rooftop':['darkorange', 1e4], 'solar':['firebrick',1e5],'onwind': ['navy',1e5]}
line_width_factor=10           
main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':['scenario A'],
          'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc':['scenario B'],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':['scenario C'],
          'Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6_2030.nc':['scenario D'],
          }
base_color='powderblue'
for i, scen in enumerate(main_scen.keys()):
   n=pypsa.Network(scen)
   loading = (n.lines_t.p0.abs().mean().sort_index()/(n.lines.s_nom_opt*n.lines.s_max_pu).sort_index()).fillna(0.)    
 
   for j, carrier in enumerate(carriers.keys()):

     p_nom_opt = n.generators.p_nom_opt[[i for i in n.generators.index if n.generators.carrier[i]==carrier]]
     p_nom_opt= (p_nom_opt.groupby([n.generators.bus.map(n.buses.location)])
             .sum())
     p_nom_max= n.generators.p_nom_max[[i for i in n.generators.index if n.generators.carrier[i]==carrier]]
     p_nom_max= (p_nom_max.groupby([n.generators.bus.map(n.buses.location)])
             .sum())
        
     iterables = [p_nom_max.index, ["p_nom_opt", "p_nom_max"]]
     shares= pd.Series(index=pd.MultiIndex.from_product(iterables, names=["bus", "attribute"]))

     for ix in p_nom_max.index:
            shares[ix]["p_nom_max"]=(p_nom_max.loc[ix]-p_nom_opt.loc[ix]).clip(min=0)   ##to show share of installed vs. max
            shares[ix]["p_nom_opt"]=p_nom_opt.loc[ix]
            
     if i==3 and carrier=='solar rooftop':
        size=carriers[carrier][1]*6
     else:
        size=carriers[carrier][1]
        
     n.plot (ax=axs[j][i],
        bus_colors={'p_nom_max':base_color,'p_nom_opt':carriers[carrier][0]},
        branch_components=["Line"],
        line_widths=0,  #GW
        line_colors=loading,
        geomap=True,
        bus_sizes=shares/size 
            )#



     axs[j][i].axis('off')
     axs[j][i].set_extent([-10.53, 28.8, 35, 70])
     if j==0: axs[j][i].set_title(main_scen[scen][0], fontsize=14)

    
"""
     if i==0: 
       colors = [base_color,carriers[carrier][0]]
       labels=[i for i in [carrier+"\npotential", "installed\n"+carrier]]
       add_legend_patch(
        axs[j][i],
        colors,
        labels,
        patch_kw=dict(edgecolor="None"),
        legend_kw=dict(bbox_to_anchor=(-0.01, 1.05), ncol=1,fontsize=10,frameon=False),)
"""
    
plt.savefig("potential_maps.png", dpi=1000)    

# 20% co2 sensitivity

In [ ]:

main_scen={'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2{}_2030.nc'
           :['-nohomegen','','$CO_{2}$ target: 95% reduction'],
          'Sensitivity-runs/postnetworks/elec_s_181_lv1.1__Co2L0p20-2H-solar+p3-dist2{}-noH2network_2030.nc'
           :['-nohomegen','','$CO_{2}$ target: 80% reduction'],}


results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name

for scen in main_scen.keys():
    for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen.format(main_scen[scen][i])
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
             scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
             scen_costs[scenario] = scen_costs[scenario].rename(index={
                          'battery':'Battery storage','battery charger':'Battery storage','battery discharger':'Battery storage',
                          'home battery':'Home battery storage','home battery charger':'Home battery storage','home battery discharger':'Home battery storage',                                                       
                           'offwind-ac':'Offshore wind','offwind-dc':'Offshore wind', 'onwind':'Onshore wind',
                          'AC':'Transmission lines'  , 'DC':'Transmission lines','ror': 'Run of river',
                          'hydro':'Hydroelectricity' , 'nuclear':'Nuclear','uranium':'Nuclear',
                          'solar':'Solar utility', 'solar rooftop':'Solar distributed',
                          'electricity distribution grid' : 'Electricity distribution grid',
                           'PHS' : 'Pumped hydro storage', 'H2':'H2 storage',
                           'SMR':'Power to gas'  , 'SMR CC':'Power to gas','oil': 'Oil and gas to power/heat',
                           'Fischer-Tropsch':'Power to liquid', 
                           'Sabatier':'Power to gas', 'helmeth':'Power to gas', 'coal' : 'Coal', 'lignite' :'Coal',
                          })  # co2 vent','co2','co2 stored',
             for ix in scen_costs[scenario].index :   
                if 'resistive heater' in ix or 'heat pump' in ix :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix : 'Power to heat',})
                if  any(w in ix for w in ['gas', 'OCGT', 'CCGT'])  :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Oil and gas to power/heat',})
                if 'biomass' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biomass',})
                if 'solar thermal' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Solar thermal',})
                if 'water tank' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Heat storage',})
                if 'biogas' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biogas',})
                #if 'biogas' in ix :scen_costs[scenario].rename(index={ix:'Biomass',})
                    
             scen_costs[scenario]=scen_costs[scenario].groupby(scen_costs[scenario].index).sum() 
            
costs=pd.DataFrame({})

preferred_order = pd.Index(
    [
        "Transmission lines",
        "Electricity distribution grid",
        "Oil and gas to power/heat",
        "Coal",
        "Power to liquid",
        "Power to gas",
        "Biomass",
        "Biogas",
        "BEV charger",
        "V2G",
        "DAC",
        "Power to heat",
        "Heat storage",
        "H2 storage",
        "H2 pipeline",
        "H2 Electrolysis",
        "H2 Fuel Cell",
        "Pumped hydro storage",
        "Hydroelectricity",
        "Run of river",
        "Nuclear",
        "Onshore wind",
        "Offshore wind",
        "Battery storage",
        "Home battery storage",
        "Solar thermal",
        "Solar utility",
        "Solar rooftop",
    ]
)


for  scenario in reversed(scen_costs.keys()) :
    costs[scenario]=pd.DataFrame(scen_costs[scenario].drop(labels =['co2 vent','co2','co2 stored'
                                                                   ]))
                                                                  #  ,'oil','uranium','lignite','coal','gas' ]) ) 
                                                        #   ['SMR', 'SMR CC','Sabatier','co2 vent','co2','co2 stored',
                                                   #    'helmeth', 'oil','uranium','lignite','coal','gas', 'CCGT','OCGT'
                                                  #  ]) ) #dropping all the variables not visible in figure
    

order = preferred_order.intersection(costs[scenario].index).append(
        costs[scenario].index.difference(preferred_order))    
costs=costs.reindex(order)   
#costs=costs.drop(labels =['process emissions','process emissions CC','Li ion'])
costs=costs.fillna(0)
results=results.fillna(0)            

In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(1, 2, figsize=(16,8)) #, sharey=True)
fig.subplots_adjust(hspace=0.5)
fig.subplots_adjust(wspace=0.5)

bar_width=0.3
index=np.arange(4) 


for i,base in enumerate(main_scen.keys()):
    for j, category in enumerate(main_scen[base][0:2]):
            scenario=base.format(category) 
            bottom=0
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
                axs[i].bar(index[j], costs[scenario][component] , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=costs[scenario][component]
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            axs[i].set_title('Scenario B\n'+main_scen[base][2]+'\n\nwithout distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            axs[i].set_ylabel('System cost\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1.2, 1.5),ncol=1,
           fontsize=20,frameon=False)

for i ,scen in enumerate (main_scen.keys()):
    fig.text(0.46*(i)+0.25, 0.45, "- "+str(round(scen_costs[scen.format(main_scen[scen][0])].sum()-
                                              scen_costs[scen.format(main_scen[scen][1])].sum(),2))
         +" bn€\n "+str(round(100*((scen_costs[scen.format(main_scen[scen][1])].sum()/
                                     scen_costs[scen.format(main_scen[scen][0])].sum())-1),2))+"%", 
         fontsize=18)

def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


arrow_style = dict(arrowstyle="simple", mutation_scale=15, ec=None)
add_arrow(axs[0], axs[0], (0.35, 0.4), (0.7, 0.4), fc="black", **arrow_style)
add_arrow(axs[1], axs[1], (0.35, 0.4), (0.7, 0.4), fc="black", **arrow_style)
#add_arrow(axs[2], axs[2], (0.4, 0.5), (0.7, 0.5), fc="black", **arrow_style)

axs[0].set_yticks(np.arange(0, 301, 50),fontsize=25)
axs[1].set_yticks(np.arange(0, 301, 50),fontsize=25)
#axs[2].set_yticks(np.arange(0, 851, 170),fontsize=25)


In [ ]:
#sensitivity for spatial resolution

import pypsa
import openpyxl
import pandas as pd
import logging
#logger.setLevel(logging.INFO)

main_scen={'Sensitivity-runs/postnetworks/elec_s_37_lv1.1__Co2L0p05-2H-solar+p3-dist2-noH2network_2030.nc': ['37'],
            'electricity_sector_only_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist2_2030.nc':['181'],  
           'Sensitivity-runs/postnetworks/elec_s_256_lv1.1__Co2L0p05-2H-solar+p3-dist2-noH2network_2030.nc':['256'],
            'Sensitivity-runs/postnetworks/elec_s_512_lv1.1__Co2L0p05-2H-solar+p3-dist2-noH2network_2030.nc':['512'],
           }
### the second network has batteries, but no rooftop

results_test=pd.DataFrame({})
transmission_expansion= [1.0]
years=[2030]

for scenario in main_scen.keys():
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
             results_test[scenario]=result#.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                           #        'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                            #       'DC']) 

In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['figure.dpi'] = 1000
plt.rcParams['hatch.linewidth'] = 1.5

fig, axs = plt.subplots(figsize=(8,4),sharex=True)
fig.subplots_adjust(hspace=0.5)
count=0
bottom=0
bar_width=0.2
hatches = ['', '//', '\\\\', 'xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']
results_=results_test.drop (labels=['Total-cost','CCGT','OCGT','H2 Fuel Cell','H2 Electrolysis',
                            'home battery charger', 'battery charger',
                           'offwind-ac','offwind-dc','ror','nuclear',
                            'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                            'gas','uranium','coal' ,'lignite','oil',
                            'DC'])
index=np.arange(len(results_.index))

preferred_order = pd.Index(
    [
        "solar",
        "battery discharger",
        "solar rooftop",
        "home battery discharger",
        "electricity distribution grid",
        "onwind",
    ]
 )
results_=results_.reindex(preferred_order)   

for k,scenario in enumerate(main_scen.keys()):
    for m, component in enumerate(results_[scenario].index):
                axs.bar(index[m]+1*k*bar_width, results_[scenario][component]/1e3 , bar_width, 
                                edgecolor='white', hatch=hatches[k], bottom=bottom,
                           color=tech_colors[component], zorder=3)
    
    #axs.set_title('Sceneario B with distributed generation and storage' ,
     #                   fontsize=14)
    axs.set_xlabel("", rotation=0)
    axs.set_xticks([], [])
    axs.tick_params(axis='y', labelsize=14)
    axs.set_ylabel("installed capacity [GW]",fontsize=14)
    axs.set_yticks(np.arange(0, 1001, 500),fontsize=20)
    
    count+=1
    
xlabels = ['Utility Solar', 'Utility battery','Distributed Solar', 'Distributed battery', 'Distribution grid','Onwind']

legend_manual=[]
for i, key in enumerate(main_scen.keys()):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', 
                                        label=main_scen[key][0]+' nodes'))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.29, .75),ncol=1,
           fontsize=14, frameon=False) 
plt.gca().add_artist(legend1)

legend_manual=[]
for i,key in enumerate(results_[scenario].index):
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=xlabels[i]))
plt.legend(handles=legend_manual[::],loc='upper left', bbox_to_anchor=(1, 0.6),ncol=1,
           fontsize=14,frameon=False)

plt.show() 

In [ ]:
#Solar rooftop difference for 181 nodes with 1 link / 2 link
import pandas as pd
import numpy as np
import pypsa
import matplotlib.pyplot as plt

Dist_factor =[1, 5] 

fig, axs = plt.subplots(4, 1, figsize=(12,5),sharex=True)
fig.subplots_adjust(hspace=0.5)
year=2030

bar_width=0.15
transmission_expansion=[1.0,'opt']
index=np.arange(4) 
colors=['yellow','gold']

results=pd.DataFrame({})
spatial_res={'37':37,'only_181':181,'only_256':256}


for i,res in enumerate(spatial_res.keys()):
             scenario=res
             nodes= spatial_res[res]
             n=pypsa.Network('electricity_sector_{}/postnetworks/elec_s_{}_lv1.0__Co2L0-2H-solar+p3-dist1_2030.nc'.format
              (res,str(nodes)))
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost' : n.objective/1e09})])
             results[scenario]=result.drop(labels = ['SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                                   'gas','H2 pipeline','uranium',
                                                   'DC']) 
            
          

In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
plt.rcParams['figure.dpi'] = 1000
plt.rcParams['hatch.linewidth'] = 1.5

fig, axs = plt.subplots(figsize=(12,5),sharex=True)
fig.subplots_adjust(hspace=0.5)
count=0
bottom=0
bar_width=0.25
hatches = ['', '//', 'xx', 'xxx','\\\\','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']
results1= results.drop(labels = ['coal' ,'lignite','oil','Total-cost','ror','CCGT','OCGT' ])
index=np.arange(len(results1.index))


for i,spatial_res in enumerate(['37','only_181','only_256']):
    scenario=spatial_res
    for m, component in enumerate(results1[scenario].index):
                axs.bar(index[m]+1.2*i*bar_width, results1[scenario][component]/1e3 , bar_width, 
                                edgecolor='white', hatch=hatches[i], bottom=bottom,
                           color=tech_colors[component], zorder=3)
    
axs.set_title('distribution grid cost factor:'+str(1)+' ,transmission expansion allowance:'+str(1),
                        fontsize=14)
axs.set_xlabel("", rotation=0)
axs.set_xticks([], [])
axs.tick_params(axis='y', labelsize=14)
axs.set_ylabel("installed capacity [GW]",fontsize=14)
axs.set_yticks(np.arange(0, 1001, 500),fontsize=20)
    

legend_manual=[]
for i, label in enumerate(['37 nodes', '137 nodes', '256 nodes']):
    legend_manual.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', label=label))
legend1=plt.legend(handles=legend_manual,loc='center right', bbox_to_anchor=(1.195, 0.9),ncol=1,
           fontsize=14, frameon=False) 
plt.gca().add_artist(legend1)

legend_manual=[]
for key in (results1[scenario].index):
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 0.8),ncol=1,
           fontsize=14,frameon=False)

plt.show() 